In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import jinja2
import os
%load_ext dotenv
%dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

## Deploy Logic App
- create docker image that will trigger on new video and preprocess images
- deploy logic app
- trigger logic app

### Create Docker image 

In [ ]:
docker_login = "<your-docker-login>"
aci_image_repo = "batchscoringdl_aci_app"

In [ ]:
%%writefile aci/requirements.txt
azure==4.0.0

In [ ]:
%%writefile aci/Dockerfile

FROM continuumio/miniconda3

RUN mkdir /app
WORKDIR /app
ADD add_images_to_queue.py /app
ADD preprocess.py /app
ADD util.py /app
ADD main.py /app
ADD requirements.txt /app

RUN conda install -c anaconda ffmpeg
RUN pip install --no-cache-dir -r requirements.txt

CMD ["python", "main.py"]

In [ ]:
# slow
!sudo docker build -t $aci_image_repo aci

In [ ]:
!sed -e "s/=\"/=/g" -e "s/\"$//g" .env > .env.docker

In [ ]:
!cat .env.docker

In [ ]:
!sudo docker run -e VIDEO="orangutan.mp4" --env-file ".env.docker" $aci_image_repo

In [ ]:
!rm .env.docker

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query 'countDetails.activeMessageCount'

In [ ]:
!sudo docker tag $aci_image_repo $docker_login/$aci_image_repo

In [ ]:
!sudo docker push $docker_login/$aci_image_repo

In [ ]:
set_key(env_path, "ACI_IMAGE", aci_image_repo)

### Deploy Logic App

Set variables we'll need for Logic App deployment:

- `logic_app` - the name of your logic app
- `aci_container_group` - the name of your ACI 
- `aci_display_name` - a display name used for your ACI deployment

In [ ]:
logic_app = "<your-logic-app>"
aci_container_group = "<your-aci-group-name>"
aci_display_name = "<your-aci-display-name>"

In [ ]:
set_key(env_path, "LOGIC_APP", logic_app)
set_key(env_path, "ACI_CONTAINER_GROUP", aci_container_group)
set_key(env_path, "ACI_DISPLAY_NAME", aci_display_name)

Using Jinja, populate the `template.logic_app.json` file and output the new file as `logic_app.json`. This file will be saved in the working directory.

In [ ]:
# use jinja to fill in variables from .env file
env = jinja2.Environment(
    loader=jinja2.FileSystemLoader('.')
)
template = env.get_template('template.logic_app.json')

e = os.environ
rendered_template = template.render(env=e)

out = open('logic_app.json', 'w')
out.write(rendered_template)
out.close()

In [ ]:
!az group deployment create \
    --name {get_key(env_path, "LOGIC_APP")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --template-file logic_app.json

Once the Logic App is deployed, go into the Azure portal and open up the ACI connector and the Azure blob connector to authenticate. 

When you open up up the Azure ACI connector, it should look like this:

![azure_aci_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_aci_connector_auth.PNG)

When you open up up the Azure blob connector, it should look like this:

![azure_blob_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_blob_connector_auth.PNG)

For both of these connectors, click on the orange bar at the top to authenticate.

Once authenticated, your Logic App should be all set up and ready to trigger the workflow.

### Trigger logic app by adding a new video to the Azure blob container

In [ ]:
!wget https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4

In [ ]:
!azcopy \
    --source orangutan.mp4 \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/orangutan2.mp4 \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --resume "."

In [ ]:
!rm orangutan.mp4